In [1]:
import torch
from torch import nn,optim
from torch.nn import functional as F
import matplotlib.pyplot as plt
import numpy as np

# 问题描述
* 已知[k,k+n]时刻的正弦函数，预测[k+t,k+n+t]时刻的正弦曲线
* 因为每个时刻曲线上的点是一个值，即embedding_dim = 1
* 如果给出50个时刻的点，即seq_len = 50
* 如果只提供一条曲线供输入则batch = 1
* 输入的shape = [seq,embedding_dim,feature_len] = [50,1,1]

In [7]:
input_size = 1
seq_len = 50
hidden_size = 16
batch_size = 1
learning_rate = 0.001
output_size = 1

In [8]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net,self).__init__()
        self.rnn = nn.RNN(
        input_size = input_size,
        hidden_size=hidden_size,
        num_layers = 2,
        batch_first = True
        )
        for p in self.rnn.parameters():#对rnn层的参数做初始化
            nn.init.normal_(p,mean = 0.0,std = 0.001)
        self.hidden_size = hidden_size
        self.linear = nn.Linear(self.hidden_size,output_size)
    def forward(self,x,prev):
        out,h = self.rnn(x,hidden_prev)
#         out:[batch,seql,outputsize]
#         因为要把输出给线性层进行处理，所以需要把batch和seq_len维度进行flatten
        out = out.view(-1,self.hidden_size)
        out = self.linear(out)#[49,1]
        out = out.unsqueeze(dim = 0)
#         out:[batch,1]->[batch,seq,1]
        return out,h

In [10]:
model = Net(input_size,hidden_size,output_size)
criteron = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)
hidden_prev = torch.zeros(2,1,16)
num_time_steps = 50

In [11]:
start = np.random.randint(3,size = 1)[0]
start

0

In [ ]:
for iter in range(6000):
    start = np.random.randint(3,size = 1)[0]
    timesteps = np.linspace(start,start+10,num_time_steps)  
    data = np.sin(timesteps)
    data  = data.reshape(num_time_steps,1)#[num_time_steps,1]
#     输入前49个点
    x = torch.tensor(data[:-1]).float().view(1,num_time_steps-1,1)#[1,49,1]
#     预测后49个点
    y = torch.tensor(data[1:]).float().view(1,num_time_steps-1,1)#[1,49,1]
    output,hidden_prev = model(x,hidden_prev)
#     print(output.shape,hidden_prev.shape)#torch.Size([1, 49, 1]) torch.Size([2, 1, 16])
    loss = criteron(output,y)
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    if iter%100 == 0:
        print('itertion:{},loss={}'.format(iter,loss.item()))
    
    
#     test
    start = np.random.randint(3,size = 1)[0]
    timesteps = np.linspace(start,start+10,num_time_steps)  
    data = np.sin(timesteps)
    data  = data.reshape(num_time_steps,1)#[num_time_steps,1]
#     输入前49个点
    x = torch.tensor(data[:-1]).float().view(1,num_time_steps-1,1)#[1,49,1]
#     预测后49个点
    y = torch.tensor(data[1:]).float().view(1,num_time_steps-1,1)#[1,49,1]
    
    predictions = []
    input= x[:,0,:]#去seq_len里面第0号数据
    input = input.view(1,1,1)
    for _ in range(x.shape[1]):
        pred,hidden_prev = model(input,hidden_prev)
        input = pred
        predictions.append(pred.detach().numpy().ravel()[0])

itertion:0,loss=0.4489847719669342
itertion:100,loss=0.04312147945165634
itertion:200,loss=0.008566171862185001


# 梯度削减

In [ ]:
loss = criteion(output,y)
optimizer.zero_grad()
loss.backward()
for p in model.parameters():
    print(p.grad.norm())
   torch.nn.utils.clip_grad_norm_(p,10)#小于等于10
optimizer。step()